# Multilevel and marginal modeling, a case study with the NHANES data

DRAFT WORK IN PROGRESS
======================

This notebook is a counterpart to the introductory regression modeling
notebook for independent data.
Here we build on the basic linear and logistic regression approaches
discussed previously in the case of independent data, and we will also
be exploring the use of some more advanced regression
approaches for data that are dependent.

We begin by importing the libraries that we will be using.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

Data can be dependent, or have a *mutilevel structure* for many reasons.
In practice, most datasets exhibit some form of dependence,
and it is arguably independent data, not dependent data, that should be treated
as the exceptional case.

Here we will reconsider the NHANES data from the perspective of dependence,
focusing in particular on dependence in the data that arises due to *clustering*
which we will define below.

First, we read in the data, as we have done before.  We retain two
additional variables here [SDMVSTRA](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#SDMVSTRA)
and [SDMVPSU](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#SDMVPSU) which will be used
below
to define the clustering structure in these data.

In [2]:
url = "https://raw.githubusercontent.com/kshedden/statswpy/master/NHANES/merged/nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

## Introduction to clustered data

One common reason that data are dependent is that the data values
were collected in clusters.  This essentially means that the population
was partitioned into groups, a limited number of these groups were
somehow selected, and then a limited number of individuals were
selected from each of the selected groups.  In a proper survey, there is
a well-planned design, in which both the groups, and the individuals within
groups, are selected randomly.  The goal in doing this is to maximize
the chances that the sample is representative of the population of interest
in all relevant ways.  But many other data sets exhibit
clustering structure, even when the data collection was not so
carefully planned.

Regardless of how the clustering in the sample arose, it is likely to be
the case that observations within a cluster are more similar to observations
in different clusters.  To make this concrete, note that clustering is
often geographic.  Data may be collected by visiting several locations, then
recruiting participants within each location.  People within a location
may share similarities, for example in demography and lifestyle, or they may
share environmental circumstances such as climate.  When
we have clustered data, it is usually advisable to account for this in the
analysis.

## Clustering structure in NHANES

The detailed process of collecting data for a study like
NHANES is very complex, so we will simplify things substantially here.
Roughly speaking, in NHANES the data are collected by selecting a limited number of
counties in the US,
then selecting subregions of these counties, then selecting people within
these subregions.  Since counties are geographically constrained, it is
expected that people within a county are more similar to each other than they
are to people in other counties.

If we could obtain the county id for each
NHANES participant, we could directly study this clustering structure.  However
for privacy reasons this information is not released with the data.  Instead, we have
access to "masked variance units" (MVUs), which are formed by combining subregions
of different counties into artificial groups that are not geographically
contiguous.  While the MVUs are not the actual clusters of the survey,
and are not truly contiguous geographic regions, they are deliberately
selected to mimic
these things, while minimizing the risk that a subject can be "unmasked"
in the data.  For the remainder of this notebook, we will treat the MVUs
as clusters, and explore the extent to which they induce correlations
in some of the NHANES variables that we have been studying.

The MVU identifiers can be obtained by combining the `SDMVSTRA` and `SDMVPSU`
identifiers, which we do next:

In [3]:
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Intraclass correlation

Similarity among observations within a cluster can be measured using a statistic
called the *intraclass correlation*, or ICC.  This is a distinct form of
correlation from Pearson's correlation.  It takes on values from 0 to 1, with
1 corresponding to "perfect clustering" -- the values within a cluster are
identical, and 0 corresponding to "perfect independence" -- the mean value
within each cluster is identical across all the clusters.

We can assess ICC using two regression techniques, marginal regression, and multilevel
regression.  We will start by using a technique called "Generalized Estimating Equations"
(GEE) to fit marginal linear models, and to estimate the ICC for the NHANES clusters.

We will first look at the ICC for systolic blood pressure:

In [4]:
model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


The estimated ICC is XXX, which is small but not negligible.  Although an ICC
is a type of correlation, its values are not directly comparable to Pearson
correlation values.  While XXX would generally be considered to be very small
as a Pearson correlation coefficient, it is not necessarily very small as an ICC.

To get a more systematic view of the ICC values induced by clustering in these
data, we calculate the ICC for a number of different variables that appear
in our analyses, either as outcomes or as predictors.

In [5]:
da["smq"] = da.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan})

for v in ["BPXSY1", "RIDAGEYR", "BMXBMI", "smq", "SDMVSTRA"]:
    model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
    result = model.fit()
    print(v, result.cov_struct.summary())

BPXSY1 The correlation between two observations in the same cluster is 0.030
RIDAGEYR The correlation between two observations in the same cluster is 0.030


BMXBMI The correlation between two observations in the same cluster is 0.030
smq The correlation between two observations in the same cluster is 0.030


SDMVSTRA The correlation between two observations in the same cluster is 0.030


To illustrate that the ICC values shown above are not consistent with a complete
absence of dependence, we simulate 10 sets of completely random data and calculate
the ICC value for each set:

In [6]:
for k in range(10):
    da["noise"] = np.random.normal(size=da.shape[0])
    model = sm.GEE.from_formula("noise ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
    result = model.fit()
    print(v, result.cov_struct.summary())

SDMVSTRA The correlation between two observations in the same cluster is -0.001


SDMVSTRA The correlation between two observations in the same cluster is -0.001
SDMVSTRA The correlation between two observations in the same cluster is -0.001


SDMVSTRA The correlation between two observations in the same cluster is -0.000


SDMVSTRA The correlation between two observations in the same cluster is -0.000


SDMVSTRA The correlation between two observations in the same cluster is 0.002
SDMVSTRA The correlation between two observations in the same cluster is 0.002


SDMVSTRA The correlation between two observations in the same cluster is 0.001


SDMVSTRA The correlation between two observations in the same cluster is 0.002


SDMVSTRA The correlation between two observations in the same cluster is -0.001


## Conditional intraclass correlation

The ICC's studied above were *marginal*, in the sense that we were looking
at whether, say, the SBP values were more similar within versus between
clusters.  To the extent that such differences are found, it may
be largely explained by demographic differences among the clusters.  For
example, we know from our previous analyses with the NHANES data
that older people have higher SBP than younger people.  Also, to some
extent some cluster may contain a slightly older set of people than
other counties.  Thus, by controlling for age, we might anticipate
that the ICC will change.  This is shown in the next analysis:

In [7]:
model = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.019
